In [ ]:
# 고객 및 음식 데이터 전체 흐름 설계도

# 1. 고객 


# 고객 아이디 입력(방식은 직접 입력 또는 안면인식)
# 입력된 고객 아이디로 고객정보 CSV 파일 읽기


### 칼로리 계산하기 & 데이터 저장하기

In [ ]:
import pandas as pd
from datetime import datetime


# 칼로리 계산하기 
def calculate_calories(base_weight, base_calories, consumed_weight):
    return base_calories * (consumed_weight / base_weight)


# 실시간 음식 테이블에 정보 저장하는 함수
def save_to_csv(file_path, food_id, name, volume, weight, calories):
    # 현재 시간 가져오기
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # 새로운 데이터 생성
    new_data = {
        "food_id": [food_id],
        "name": [name],
        "volume": [volume],
        "weight": [weight],
        "calories": [calories],
        "timestamp": [timestamp]
    }

    # 새로운 데이터프레임 생성
    new_df = pd.DataFrame(new_data)

    try:
        # 기존 파일 읽기 (있으면 데이터 추가)
        existing_df = pd.read_csv(file_path)
        updated_df = pd.concat([existing_df, new_df], ignore_index=True)
    except FileNotFoundError:
        # 파일이 없으면 새 파일 생성
        updated_df = new_df

    # CSV 파일 저장
    updated_df.to_csv(file_path, index=False)
    print(f"{name} 정보가 {file_path}에 저장되었습니다.")


# 카메라와 저울로부터 받은 데이터를 칼로리 계산한 뒤 저장하는 함수
# 위 두가지 통합
def process_food_data(camera_food_id, measured_weight, measured_volume, csv_file_path):
    # CSV 파일 경로 (음식 정보)
    food_data_path = 'food_data.csv'

    # CSV 파일 읽기
    food_data = pd.read_csv(food_data_path)

    # food_id에 해당하는 음식 정보 가져오기
    food_info = food_data[food_data['food_id'] == camera_food_id]

    if not food_info.empty:
        # 음식 정보 추출
        name = food_info['name'].values[0]
        base_weight = food_info['base_weight'].values[0]
        base_calories = food_info['base_calories'].values[0]

        # 칼로리 계산
        calculated_calories = calculate_calories(base_weight, base_calories, measured_weight)

        # CSV 파일에 저장
        save_to_csv(csv_file_path, camera_food_id, name, measured_volume, measured_weight, calculated_calories)
    else:
        print(f"food_id {camera_food_id}에 해당하는 음식 정보를 찾을 수 없습니다.")

# 카메라에서 반환된 food_id
camera_food_id = 1

# 저울 측정값 (무게: g)
measured_weight = 180

# 부피 측정값 (예: ml 또는 cm³)
measured_volume = 200

# 저장할 CSV 파일 경로
output_csv_path = 'real_time_food_info.csv'

# 실시간 음식 데이터 처리 및 저장
process_food_data(camera_food_id, measured_weight, measured_volume, output_csv_path)


### 고객 정보 가져오기 

In [9]:
import pandas as pd

# 고객 id로 csv파일에서 고객정보를 필터링하여 반환
# 고객 정보를 딕셔너리로 반환
def get_customer_info(customer_id, csv_file_path):
    try:
        # CSV 파일 읽기
        customer_data = pd.read_csv(csv_file_path, dtype = {'연락처' : object})

        # 고객 ID로 데이터 검색(필터링)
        customer_info = customer_data[customer_data['고객아이디'] == customer_id]

        # 정보가 존재한다면
        if not customer_info.empty:
            # 고객 정보를 딕셔너리로 변환
            # orient : 데이터프레임을 딕셔너리로 변환하는 형식을 지정
            # orient='records' -> 각 행을 딕셔너리로 표현한 리스트를 반환
            return customer_info.to_dict(orient='records')[0]
        else:
            print(f"고객 ID {customer_id}에 해당하는 정보를 찾을 수 없습니다.")
            return None
    except FileNotFoundError:
        print(f"CSV 파일 {csv_file_path}이(가) 존재하지 않습니다.")
        return None

# 실행
csv_file_path = 'FOOD_DB/food_project_user_info.csv'  # 고객 정보가 저장된 CSV 파일 경로

# 예시코드에서는 INPUT으로 고객 아이디를 입력받지만 
# 데모시에는 얼굴인식기능으로 인식받은 고객 아이디 
customer_id = input("고객 ID를 입력하세요: ")  # 고객 ID 입력(sesac1)

# 고객 정보 검색
customer_info = get_customer_info(customer_id, csv_file_path)

if customer_info:
    print("고객 정보를 찾았습니다:")
    for key, value in customer_info.items():
        print(f"{key}: {value}")


고객 정보를 찾았습니다:
고객아이디: sesac1
고객명: 홍유경
생년월일: 19251022
성별: F
연락처: 01012345678
주소: 서울시 송파구 가락동
키: 170.2
몸무게: 55.2
운동량: E


### 만 나이 계산

In [20]:
import datetime

# 생년월일을 기준으로 만 나이 계산하는 함수
def calculate_age(birth_date):

    # 생년월일 분리
    birthYear = int(birth_date[:4])
    birthMonth = int(birth_date[4:6])
    birthDay = int(birth_date[6:])
    
    # 오늘 날짜 가져오기
    today = datetime.date.today()
    thisYear = today.year
    thisMonth = today.month
    thisDay = today.day

    # 생일이 지났는지 확인
    if thisMonth > birthMonth or (thisMonth == birthMonth and thisDay >= birthDay):
        age = thisYear - birthYear
    else:
        age = thisYear - birthYear - 1
    
    return age

birth_date = str(customer_info['생년월일'])
age = calculate_age(birth_date)

print(age)


99


### 검색한 고객정보로 BMI/권장식사량 계산하기

In [ ]:
# customer_info

weight = customer_info['몸무게']
print(weight)
height = customer_info['키']
print(height)
exercise_score = customer_info['운동량']
print(exercise_score)
sex = customer_info['성별']
print(sex)
birth_date = str(customer_info['생년월일'])
age = calculate_age(birth_date)
print(age)


BMR = ((10 * weight) + (6.25 * height) - (5 * age))



if sex == 'F':
    BMR_woman = BMR - 161
    print(BMR_woman)
else: 
    BMR_man = BMR + 5
    print(BMR_man)




55.2
170.2
E
F
99
959.75


In [ ]:
# 기초대사율과 활동대사량을 곱하여 하루 권장 식사량을 계산  
def calculate_bmr(weight, height, age, sex, exercise_score):

    # 기본 BMR 계산식
    bmr = (10 * weight) + (6.25 * height) - (5 * age)

    # 성별에 따른 조정
    if sex == 'F':  # 여성
        bmr -= 161
    elif sex == 'M':  # 남성
        bmr += 5
    else:
        raise ValueError("성별은 'M' 또는 'F'로 입력해야 합니다.")

    # 운동량에 따른 조정값 설정
    activity_multiplier = {
        'A': 1.9,    # 매우 높은 활동량
        'B': 1.725,  # 높은 활동량
        'C': 1.55,   # 보통 활동량
        'D': 1.375,  # 적은 활동량
        'E': 1.2     # 거의 활동하지 않음
    }

    # 유효한 운동량인지 확인
    if exercise_score not in activity_multiplier:
        raise ValueError("운동량은 'A', 'B', 'C', 'D', 'E' 중 하나여야 합니다.")
    
    # 운동량 곱하기 값 적용
    adjusted_bmr = bmr * activity_multiplier[exercise_score]
    return adjusted_bmr

# 예제 실행
weight = customer_info['몸무게']
height = customer_info['키']
age = calculate_age(birth_date)
sex = customer_info['성별']
exercise_score = customer_info['운동량']

adjusted_bmr_value = calculate_bmr(weight, height, age, sex, exercise_score)
one_meal_value = 3/adjusted_bmr_value
print(f"운동량을 고려한 조정된 BMR: {adjusted_bmr_value:.2f} kcal")
print(f"한끼 권장 식사량: {one_meal_value:}")

운동량을 고려한 조정된 BMR: 1151.70 kcal


### csv 파일 읽어오기

In [11]:
import pandas as pd

food_data = pd.read_csv('C:/Users/Sesame/Downloads/food_project_음식데이터.csv', dtype = {'음식 label' : object})
# 각 열의 데이터 타입을 확인하고 필요하다면 타입변환을 해준다. 
# 음식 레이블값은 0으로 시작하는 경우 0을 생략해선 안되므로 int형이었던것을 object 형태로 변환해줌 
# food_data = pd.read_csv('C:/Users/Sesame/Downloads/food_project_음식데이터.csv')

print(food_data.dtypes)
print(food_data)

음식 아이디         int64
음식 label      object
음식명           object
중량(g)          int64
에너지(kcal)    float64
탄수화물(g)      float64
당류(g)        float64
지방(g)        float64
단백질(g)       float64
칼슘(mg)       float64
인(mg)        float64
나트륨(mg)      float64
칼륨(mg)       float64
마그네슘(mg)      object
철(mg)        float64
아연(mg)       float64
콜레스테롤(mg)    float64
트랜스지방(g)       int64
dtype: object
    음식 아이디  음식 label    음식명  중량(g)  에너지(kcal)  탄수화물(g)  당류(g)  지방(g)  단백질(g)  \
0        0  01011001     쌀밥    210     334.80    73.71   0.00   0.45    5.76   
1        1  04017001   된장찌개    400     147.06    15.98   0.00   5.27   11.71   
2        2  06012004    떡갈비    250     762.99    26.61  12.49  51.58   43.09   
3        3  07014001   달걀말이    100     172.24     4.67   0.00  11.23   12.04   
4        4  11013007  시금치나물     50      37.51     3.82   0.00   2.37    2.07   
5        5  12011008   배추김치     50      18.43     4.16   0.50   0.30    1.15   
6        6  01012006    흑미밥    200     318.00   

In [ ]:
# 아이디가 3인 음식의 칼로리 계산해보기(아이디는 현재 int형임)
# 양은 150g이라고 가정함 


food_id = int(input()) #카메라로부터 값을 받아옴
consumed_weight = int(input())   #저울로부터 값을 받아옴
file_path = 'C:/Users/Sesame/Downloads/food_project_음식데이터.csv'

# 기준 중량과 기준 칼로리에 따라 칼로리를 계산하는 함수 
def calculate_calories(base_calories, base_weight, consumed_weight):
    return base_calories * (consumed_weight/base_weight)


# food_data에서 base_calories와 base_weight를 가져오는 함수를 미리 만들어두자 
def get_base_calories_base_weight(food_id,file_path):
    food_data = pd.read_csv(file_path)
    food_info = food_data[food_data['음식 아이디'] == food_id]
    base_calories = food_info['에너지(kcal)']
    base_weight = food_info['중량(g)']
    food_name = food_info['음식명']

    return base_calories, base_weight, food_name
    # return food_info


base_calories, base_weight, food_name = get_base_calories_base_weight(food_id, file_path)
# food_info = get_base_calories_base_weight(food_id, file_path)
print(f'음식명 : {food_name}')
print(f'기준 칼로리 : {base_calories}')
print(f'기준 중량 : {base_weight}')
# print(food_info)

calorie = calculate_calories(base_calories, base_weight, consumed_weight)
print(f'배식량 : {consumed_weight}')
print(f'칼로리 : {calorie}')


음식명 : 7    미역국
Name: 음식명, dtype: object
기준 칼로리 : 7    50.17
Name: 에너지(kcal), dtype: float64
기준 중량 : 7    500
Name: 중량(g), dtype: int64
배식량 : 150
칼로리 : 7    15.051
dtype: float64


### 음식 정보의 영양정보 출력하기 

In [ ]:
# 기준 중량 대비 배식한 중량의 영양 정보를 출력할 수 있다. 
# 칼로리 계산법과 같음 

### 계산된 칼로리를 실시간 음식정보 테이블에 저장하기 

### 고객 식단상세에 로그아이디 생성하여 저장하기 

In [ ]:

import pandas as pd
from datetime import datetime

def generate_customer_log(customer_id, date, log_file_path, additional_data=None):
    """
    특정 고객의 당일 로그기록 ID를 생성하고 데이터를 저장.
    
    Parameters:
        customer_id (str): 고객 ID.
        date (str): 로그 생성 기준 날짜 (형식: YYYYMMDD).
        log_file_path (str): 로그 데이터를 저장한 CSV 파일 경로.
        additional_data (dict): 로그에 추가로 저장할 데이터 (옵션).
    
    Returns:
        str: 새로 생성된 로그기록 ID.
    """
    try:
        # 기존 로그 데이터 읽기
        logs = pd.read_csv(log_file_path)
    except FileNotFoundError:
        # 파일이 없으면 새로운 데이터프레임 생성
        logs = pd.DataFrame(columns=['log_id', 'customer_id', 'date', 'other_info'])

    # 해당 고객과 날짜의 로그 추출
    existing_logs = logs[(logs['customer_id'] == customer_id) & (logs['date'] == date)]

    # 해당 날짜 및 고객의 최대 일련번호 계산
    if not existing_logs.empty:
        max_number = existing_logs['log_id'].apply(lambda x: int(x.split('_')[-1])).max()
    else:
        max_number = 0

    # 새 일련번호 생성
    new_number = max_number + 1
    new_log_id = f"{date}_{new_number}"

    # 새로운 로그 데이터 추가
    new_log_entry = {
        'log_id': new_log_id,
        'customer_id': customer_id,
        'date': date,
        'other_info': additional_data if additional_data else "기본 로그 내용"
    }
    logs = logs.append(new_log_entry, ignore_index=True)

    # 수정된 로그 데이터를 CSV 파일에 저장
    logs.to_csv(log_file_path, index=False)
    print(f"새 로그기록 ID 생성: {new_log_id}")
    return new_log_id

# 테스트 실행
today_date = datetime.now().strftime('%Y%m%d')  # 오늘 날짜 (YYYYMMDD 형식)
log_file_path = "customer_logs.csv"  # 로그 기록 CSV 파일 경로
customer_id = "C001"  # 테스트 고객 ID

# 추가 로그 데이터 (옵션)
additional_data = "오늘의 식단 로그"

# 로그 생성
generate_customer_log(customer_id, today_date, log_file_path, additional_data)
